In [1]:
from Models import WGAN_GP
# IPython 확장 기능 로드
%load_ext autoreload

# 모든 모듈 자동 재로드 설정
%autoreload 2
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import Models as md
import base_import as bi
from base_import import calculate_fid, MINMAX_CUSTOM



In [ ]:
df = pd.read_csv('data/clean_for_nex2.csv')
df['label'].unique()

In [ ]:
df['label'].apply(lambda x: 1 if x != "BENIGN" else 0)

In [ ]:
# 데이터 로드 및 DataLoader 정의
ds = bi.DATASETS('data/clean_for_nex2.csv')

In [ ]:
ds.df.drop(["destination_port"], axis=1, inplace=True)
ds.df

In [ ]:
ds.df.isnull().values.any()

In [ ]:
ds.df['label'].value_counts()

In [ ]:
label_mapping = ds.df['label'].values
unique_val = np.array(label_mapping)
np.unique(unique_val)

In [ ]:
tmp_df = ds.df.copy()
labelencoder = LabelEncoder()
tmp_df.iloc[:,-1] = labelencoder.fit_transform(ds.df.iloc[:,-1])
tmp_df['label'].unique()

In [ ]:
tmp_df['label'].value_counts()

In [ ]:
tmp_df = bi.Auto_Feature_Engineering(tmp_df)
ds.setDF(tmp_df)

In [ ]:
ds.df.shape

In [ ]:
indices_to_keep = ~ds.df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
df2 = ds.df[indices_to_keep]

In [ ]:
ds.features

In [ ]:
ds.labels

In [ ]:
from cProfile import label

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

import Models as md
import base_import as bi

if not tf.config.list_physical_devices('GPU'):
    print("No GPU found. Please make sure that a GPU is available.")
else:
    print("GPU is available: ", tf.config.list_physical_devices('GPU'))
# 데이터 로드 및 DataLoader 정의
# ds = bi.DATASETS('data/clean_sampled30.csv')
ds = bi.DATASETS('data/sampled_cic2018_data.csv')

In [ ]:

#preprocesssing
#ds.df.drop(["destination_port"], axis=1, inplace=True)
ds.df.drop(["dst_port"], axis=1, inplace=True)
if 'Unnamed: 0' in ds.df.columns:
    ds.df.drop(['Unnamed: 0'], axis=1, inplace=True)
    
#label mappings
label_mapping = ds.df['label'].values
unique_val = np.array(label_mapping)
tmp_df = ds.df.copy()
labelencoder = LabelEncoder()
tmp_df.iloc[:,-1] = labelencoder.fit_transform(ds.df.iloc[:,-1])

# feature engineering
tmp_df = bi.Auto_Feature_Engineering(tmp_df)
ds.setDF(tmp_df)
indices_to_keep = ~ds.df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
df2 = ds.df[indices_to_keep]


In [ ]:
ds.df['label'] = pd.to_numeric(ds.df['label'], errors='coerce').astype('int')

In [ ]:

# Hyperparameters 설정
latent_dim = 32
data_dim = 21 # 데이터 컬럼 수
batch_size = 128
num_epochs = 5
learning_rate = 0.0001
n_critic = 5
label_classes = len(ds.df['label'].unique())

# Refresh features, scaler features, labels
ds.refreshDataset()

custom_mx = MINMAX_CUSTOM()

# dataset = tf.data.Dataset.from_tensor_slices((ds.scaler_features, ds.labels)).batch(batch_size)
dataset = tf.data.Dataset.from_tensor_slices((custom_mx.scale_transforms(ds.features)
                                              , ds.df['label'])).batch(batch_size).prefetch(tf.data.AUTOTUNE)



In [ ]:
# GAN 모델 초기화 및 학습
gan = md.WGAN_GP(latent_dim, data_dim, batch_size, num_epochs, learning_rate, n_critic, label_classes)
gan.train(dataset)

In [ ]:
vinila_gan = md.VanillaGAN(latent_dim, data_dim, batch_size, num_epochs, learning_rate, n_critic)
vinila_gan.train(dataset)

In [ ]:
ds_minor = bi.DATASETS('data/minor_attack.csv')

In [ ]:
# 생성할 샘플 수 설정
num_samples = 1000  # 평가를 위해 충분히 많은 샘플 생성

# 노이즈 및 레이블 생성
random_latent_vectors = tf.random.normal(shape=(num_samples, latent_dim))
random_labels = np.random.randint(0, label_classes, size=(num_samples, 1))

# Generator를 사용하여 가짜 데이터 생성
wgen_pred = gan.generator.predict([random_latent_vectors, random_labels])
wgen_data = gan.generator([random_latent_vectors, random_labels])
# vgen_pred = vinila_gan.generator.predict([random_latent_vectors, random_labels])
# vgen_data = vinila_gan.generator([random_latent_vectors, random_labels])

wgen_data#, vgen_data

In [ ]:
inv_wgen = custom_mx.inverse_transform(wgen_data)
inv_wgen

In [ ]:
wgen_dis_pred = gan.discriminator.predict(wgen_data)
#vgen_dis_pred = vinila_gan.discriminator.predict(vgen_data)

In [ ]:
wgen_fid = calculate_fid(ds.scaler_features, ds.scaler.fit_transform(wgen_pred))
#vgen_fid = calculate_fid(ds.scaler_features, ds.scaler.fit_transform(vgen_pred))

print(f"wgen fid: {wgen_fid:.2f},")
#print(f"vgen fid: {vgen_fid:.2f}")

In [ ]:
ds.features

In [ ]:
a = ds.scaler.inverse_transform(wgen_data)
a

In [ ]:
import matplotlib.pyplot as plt

def plot_column_distribution(df, column_index=0):
    # 0번째 컬럼 데이터 추출
    column_name = df.columns[column_index]
    column_data = df[column_name]

    # 히스토그램 그리기
    plt.hist(column_data, bins=30, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title(f'Distribution of Column: {column_name}')
    plt.grid(axis='y', linestyle='--')
    plt.show()

# 예시 사용
# tmp_df는 MINMAX_CUSTOM 클래스에서 반환된 데이터프레임이라고 가정
# plot_column_distribution(tmp_df)
for i, d in enumerate(ds.df.columns):
    plot_column_distribution(pd.DataFrame(a), column_index=i)
    plot_column_distribution(pd.DataFrame(ds.df), i)
    

In [ ]:
ds.df

##########################
라벨 하나씩 학습하기
########################## 

In [ ]:
label_code = pd.read_csv('data/classes/label_code_count.csv')


In [ ]:
ds.df[ds.labels == 'BENIGN']

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

dfs = bi.DATASETS('data/clean1.csv')

#label mappings
label_mapping = dfs.df['label'].values
unique_val = np.array(label_mapping)
tmp_df = dfs.df.copy()
labelencoder = LabelEncoder()
tmp_df.iloc[:,-1] = labelencoder.fit_transform(dfs.df.iloc[:,-1])

# feature engineering
tmp_df = bi.Auto_Feature_Engineering(tmp_df)
dfs.setDF(tmp_df)
indices_to_keep = ~dfs.df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
label_df = dfs.df[indices_to_keep]

# 라벨별로 데이터 나누기
labels = label_df['label'].unique()
label_datasets = {label: label_df[label_df['label'] == label].drop(columns=['label']) for label in labels}
gans = {}
# 각 라벨별로 VanillaGAN 학습
for label, dataset in label_datasets.items():
    print(f"Training GAN for label: {label}")

    # 스케일링 수행
    # scaler = MINMAX_CUSTOM()
    # scaled_data = scaler.scale_transforms(dataset)
    scaler = MinMaxScaler().fit(dataset)
    scaled_data = scaler.transform(dataset)
    # 데이터셋 준비
    scaled_features = pd.DataFrame(scaled_data).values
    batch_size = 128  # 적절한 배치 크기로 설정
    tf_dataset = tf.data.Dataset.from_tensor_slices((scaled_features, [label] * len(scaled_features))).batch(batch_size)

    # GAN 초기화 및 학습
    gan = md.WGAN_GP(latent_dim=32, data_dim=scaled_features.shape[1], batch_size=batch_size, num_epochs=5, learning_rate=0.0002, label_classes=len(labels))
    gan.train(tf_dataset)
    # 학습된 GAN 모델을 딕셔너리에 저장
    gans[label] = {
        'gan': gan,
        'scaler': scaler  # 생성 후 역변환에 사용할 스케일러도 함께 저장
    }


                        feature  normalized_importance  cumulative_importance
0              destination_port               0.121000               0.121000
1        init_win_bytes_forward               0.091667               0.212667
2       init_win_bytes_backward               0.062333               0.275000
3             fwd_header_length               0.050667               0.325667
4   total_length_of_fwd_packets               0.041667               0.367333
..                          ...                    ...                    ...
62                bwd_urg_flags               0.000000               1.000000
61               ece_flag_count               0.000000               1.000000
60          fwd_header_length.1               0.000000               1.000000
67         avg_fwd_segment_size               0.000000               1.000000
77            fwd_avg_bulk_rate               0.000000               1.000000

[78 rows x 3 columns]
                   drop_feature          

In [12]:
for a in gans:

    # 랜덤 노이즈 생성 (예: 32개의 샘플, 차원은 latent_dim과 동일하게 설정)
    num_samples = 32
    random_latent_vectors = tf.random.normal(shape=(num_samples, gan.latent_dim))
    
    # 원하는 라벨을 지정 (예: 0번 라벨을 가진 데이터를 생성하고 싶을 때)
    label_value = a
    random_labels = np.full((num_samples, 1), label_value, dtype=np.int32)  # 모든 라벨을 지정된 값으로 설정
    
    # Generator를 통해 가짜 데이터 생성
    gans[a]['gendata'] = gans[a]['gan'].generator([random_latent_vectors, random_labels])
    


array([[8.37186236e-01, 1.33333321e-07, 4.55046005e-06, ...,
        0.00000000e+00, 9.99999780e-01, 0.00000000e+00],
       [8.40070192e-01, 1.01666657e-06, 0.00000000e+00, ...,
        3.92150879e-03, 9.99999780e-01, 0.00000000e+00],
       [8.40085451e-01, 5.41666617e-07, 0.00000000e+00, ...,
        3.92150879e-03, 9.99999780e-01, 0.00000000e+00],
       ...,
       [8.85481041e-01, 7.91666594e-07, 4.55046005e-06, ...,
        1.52587891e-05, 9.99999803e-01, 0.00000000e+00],
       [8.08728161e-04, 8.73873253e-03, 2.27523003e-05, ...,
        0.00000000e+00, 9.99999780e-01, 0.00000000e+00],
       [8.08728161e-04, 7.91266594e-04, 1.36513802e-05, ...,
        0.00000000e+00, 9.99999780e-01, 0.00000000e+00]])

In [84]:
gans[0]['gendata']

<tf.Tensor: shape=(32, 24), dtype=float32, numpy=
array([[-9.36483592e-03,  9.83344764e-03, -9.85633698e-04,
        -4.15673247e-04,  4.77431994e-03,  5.78949368e-03,
         6.08228706e-03,  1.11345155e-03,  1.10196896e-01,
         2.71912501e-03, -3.33085714e-04, -3.48074082e-03,
         1.52875669e-04,  2.42739450e-03,  9.98015821e-01,
         1.00061357e+00,  1.56216798e-02,  9.18979105e-03,
         1.95799209e-03, -1.89056173e-02, -7.53495470e-03,
        -4.41823155e-04,  9.99997854e-01,  5.71534969e-03],
       [-1.41404569e-04,  3.73560935e-03, -2.20302973e-04,
        -1.13939214e-03,  3.90469399e-03,  8.29044357e-03,
         7.58562516e-03, -4.16096300e-04,  1.10471353e-01,
         3.09371389e-03,  3.83117498e-04, -1.82954594e-04,
        -1.70341367e-03,  5.24738338e-03,  9.98912096e-01,
         9.94976044e-01,  4.83281165e-03,  1.20065799e-02,
        -3.65505740e-03, -1.90941170e-02,  2.01851502e-03,
        -3.79120931e-04,  1.00230181e+00,  5.40269539e-04],
    

In [13]:
for a in gans:
    gans[a]['invd'] = gans[a]['scaler'].inverse_transform(gans[a]['gendata'])

In [85]:
wgen_fid = calculate_fid(gans[2]['scaler'].transform(label_datasets[2]) , gans[2]['gendata'])

In [86]:
wgen_fid


0.5139116997669038

In [14]:
gans[0]['invd']

array([[-6.13724522e+02,  1.18000082e+06, -2.15600890e+02,
        -5.36218488e+03,  1.18498621e+02,  1.34605728e+01,
         2.84164451e+01,  2.17457088e+01, -4.02083877e+06,
         3.26282037e+05, -3.99842904e+04, -2.94480870e+05,
         1.83450803e+04,  2.91287340e+05, -5.92791485e+07,
         6.50083798e+06,  4.68650395e+04,  1.83795821e+04,
         1.95799209e-03, -1.89056173e-02, -4.94810791e+02,
        -2.99553223e+01, -1.01399976e+03,  6.85841963e+05],
       [-9.26694844e+00,  4.48260163e+05, -4.74133407e+01,
        -1.46981587e+04,  9.69145049e+01,  1.92752813e+01,
         3.54400408e+01, -8.12636074e+00, -3.38080496e+06,
         3.71232708e+05,  4.59601051e+04, -1.54784946e+04,
        -2.04409640e+05,  6.29686005e+05, -3.04039634e+07,
         4.14678465e+05,  1.44984350e+04,  2.40131598e+04,
        -3.65505740e-03, -1.90941170e-02,  1.31285400e+02,
        -2.58460693e+01,  1.23591374e+06,  6.48323447e+04],
       [ 5.54490133e+04, -9.32609113e+05, -6.66793313e

In [62]:
tocsvG = {}
for a in gans:
    tocsvG[a] = gans[a]['invd'].copy()
    tocsvG[a] = pd.DataFrame(tocsvG[a])
    tocsvG[a][tocsvG[a].shape[1]] = a
    tocsvG[a].to_csv(f'data/generated/generated_{a}.csv', index=False)
    

In [59]:
tocsvG[0]

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-613.724522,1.180001e+06,-215.600890,-5362.184881,118.498621,13.460573,28.416445,21.745709,-4.020839e+06,3.262820e+05,...,6.500838e+06,46865.039505,18379.582092,0.001958,-0.018906,-494.810791,-29.955322,-1.014000e+03,6.858420e+05,0
1,-9.266948,4.482602e+05,-47.413341,-14698.158670,96.914505,19.275281,35.440041,-8.126361,-3.380805e+06,3.712327e+05,...,4.146785e+05,14498.434961,24013.159797,-0.003655,-0.019094,131.285400,-25.846069,1.235914e+06,6.483234e+04,0
2,55449.013277,-9.326091e+05,-666.793313,-29744.971404,234.444259,-10.125913,42.260255,347.725263,1.483431e+06,-1.074543e+06,...,-1.361390e+06,33362.790942,11750.732549,0.006852,0.169864,780.388550,230.772949,5.130377e+06,8.035532e+05,0
3,-345.943452,3.993157e+07,-2758.955546,144359.866250,244.193860,33.858169,-20.393393,1510.492968,4.409269e+07,2.790558e+06,...,7.092435e+07,10991.737247,8871.961385,1.009251,0.016532,8198.359375,864.608276,-1.005986e+07,2.391713e+07,0
4,5751.404427,4.537515e+06,-289.745169,5037.742530,211.829720,-6.405484,-11.563535,-231.297456,8.225548e+06,5.215438e+05,...,1.097019e+07,-14406.943694,81365.168095,0.002685,0.995078,1179.154907,-513.578003,7.142498e+05,-4.347589e+06,0
5,-378.428308,7.108174e+05,-60.901414,-13915.532688,99.791480,17.663314,38.787859,-7.485890,-2.722198e+06,4.345412e+05,...,1.809485e+06,13216.998428,20672.576502,-0.004634,-0.020603,310.321045,-9.534790,1.482698e+06,4.568332e+05,0
6,59917.741964,4.208433e+05,-551.526832,-35598.522378,210.905704,-2.935256,77.511870,243.201507,1.021273e+07,-7.965843e+05,...,-9.238352e+04,61476.213858,16838.453710,-0.000084,-0.023982,1462.325928,369.421265,7.412578e+05,8.030080e+05,0
7,71.977906,4.057275e+05,-27.145384,-13699.243544,91.126570,18.723559,35.073507,-3.986469,-3.203756e+06,3.520609e+05,...,6.028319e+05,13486.655429,22213.183343,-0.001909,-0.021133,288.616211,-68.786133,8.757858e+05,5.857006e+04,0
8,-98.006805,4.483616e+05,-55.986664,-14641.668554,102.580506,18.419068,38.159559,-13.546921,-2.627262e+06,4.089237e+05,...,3.796088e+05,15666.341409,22984.296083,-0.005708,-0.018954,383.100586,-6.619019,1.612298e+06,1.819385e+05,0
9,49158.937383,1.505291e+05,-987.783758,85663.163848,111.459303,2.531186,-12.848374,-90.356999,-8.497305e+06,7.553613e+05,...,-2.454083e+06,-39964.925498,-3621.093929,-0.001970,1.015557,-67.656494,135.296753,5.775049e+06,-1.259282e+06,0


In [ ]:
# Classifier를 사용하여 생성된 데이터의 라벨을 예측
predicted_labels = gan.classifier.predict(tocsvG)

# 예측된 라벨 확인
predicted_labels_class = np.argmax(predicted_labels, axis=1)  # 라벨을 one-hot에서 원래 라벨로 변환
print(f"Predicted labels: {predicted_labels_class}")


from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
# 3. 성능 평가 - Accuracy, F1-Score, Precision, Recall

# 라벨을 원래 라벨 형식으로 변환 (실제 라벨과 비교)
true_labels = labels.flatten()  # 라벨의 차원 축소 (1D로 변환)

# Accuracy 계산
accuracy = accuracy_score(true_labels, predicted_labels_class)

# F1-Score 계산
f1 = f1_score(true_labels, predicted_labels_class, average='weighted')

# Precision 계산
precision = precision_score(true_labels, predicted_labels_class, average='weighted')

# Recall 계산
recall = recall_score(true_labels, predicted_labels_class, average='weighted')

conf_matrix = confusion_matrix(true_labels, predicted_labels_class)
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix)  # False Positives
TN = conf_matrix.sum() - (FP + conf_matrix.sum(axis=1) - np.diag(conf_matrix))  # True Negatives
FAR = FP / (FP + TN)

# 성능 지표 출력
print(f"False Alarm Rate (FAR): {np.mean(FAR)}")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")